## Домашнее задание 5

В данном домашнем задании Вам предстоит реализовать автоматическое исправление опечаток в запросах пользователей. 

### 1. Датасет
Для оценки качества алгоритма исправления опечаток, Вам предоставляется файл `queries.tsv.gz`. В каждой строке файла записаны два запроса – исходный и исправленный. Для простоты, оба запроса будут иметь одинаковое количество слов и отличаться незначительно. Зачастую исходный и исправленный запрос совпадают, что означает что исправлять такой запрос не требуется.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from typing import List, Tuple, Generator, Callable

Query = str
Sentence = str
Filename = str
Word = str
Queries = List[Tuple[Query, Query]]

In [3]:
from termcolor import colored
import difflib

def diff_queries(original: Query, fixed: Query) -> Query:
    result = ''
    for pos, d in enumerate(difflib.ndiff(original, fixed)):
        if d[0] == '+':
            result += colored(d[2], 'green')
        elif d[0] == '-':
            result += colored(d[2], 'red')
        else:
            result += d[2]
    return result

print(diff_queries("lake compond the park", "lake compound the park"))
print(diff_queries("traditional chothes", "traditional clothes"))
print(diff_queries("jack sparrow", "captain jack sparrow"))

lake compound the park
traditional chlothes
captain jack sparrow


In [4]:
import gzip

def load_queries(fn: Filename) -> Queries:
    result = []
    with gzip.open(fn, 'rt', encoding='utf8') as inp:
        for line in inp:
            original, fixed = line.rstrip('\n').split('\t')
            result.append((original, fixed))
    return result

queries = load_queries("./drive/MyDrive/queries.tsv.gz")
print(f'Loaded {len(queries)} queries\n')
for original, fixed in queries[10:20]:
    print(diff_queries(original, fixed))

Loaded 102436 queries

embearirassing red carpet moments
grants for rural areas florirda
the home hdepot merchandising
delaware motorcycle inspection requirements
highland park hospital gastric biypass surgery
grand theift auto
windward community college
my credit reports
strack intermediate school
mongol empire political system


In [5]:
queries_sample = [
    ("grand theift auto", "grand theft auto"),
    ("belarus longitude and latitdue", "belarus longitude and latitude"),
    ("search for poeoms", "search for poems"),
    ("large guacolmoi dip restaurtant price", "large guacamole dip restaurant price"),
    ("texas chainsaw mascurer", "texas chainsaw massacre"),
    ("royal trump subtitle", "royal tramp subtitle"),
    ("florida fiberglass polls", "florida fiberglass pools"),
    ("how to make a calender", "how to make a calendar"),
    ("university of south caroline", "university of south carolina"),
    ("maureen mcdonald in virginia", "maureen mcdonnell in virginia"),
]

Для составления словаря и обучения языковых моделей Вам предоставляется небольшой корпус текста, неслучайная выборка из большой английской википедии в файле `train.bz2`. Этот файл содержит примерно 5 млн строк или 80 млн слов. Каждая строка – одно предложение без знаков препинания.
Использование других словарей и корпусов запрещено.

In [6]:
import bz2
from tqdm import tqdm

def read_huge_corpus(fn: Filename) -> Generator[Sentence, None, None]:
    with bz2.open(fn, 'rt', encoding='utf8') as inp:
        for line in tqdm(inp):
            yield line.rstrip('\n')

for li, line in enumerate(read_huge_corpus("./drive/MyDrive/train.bz2")):
    print(line)
    if li == 10:
        break

10it [00:00, 259.25it/s]

gol neshin
mitochondrial dna depletion syndrome mds or mdds is any of a group of autosomal recessive disorders that cause a significant drop in mitochondrial dna in affected tissues
following the relegation of sc freiburg in 2005 he was on the verge of signing for metalurg donetsk but instead he accepted a contract with vfl wolfsburg
the first issue for geometers is what kind of geometry is adequate for a novel situation
cedar grove was formerly a stage and freight stop
regular bus service runs from bhubaneswar to niali which is away
later they were also known for the cream wafer biscuits
strabomantis cornutus
gtk+ scene graph kit gsk was initially released as part of gtk+ 3.90 in march 2017 and is meant for gtk-based applications that wish to replace clutter for their ui
the match took place on 10 april 1906 at the hipódromo madrid
the brothers came from fresno california


### 2. Поиск близких слов
Требуется научится быстро находить список из сотни слов, которые незначительно отличаются от заданного слова.

Не стоит перебирать все слова словаря – займёт слишком много времени.

Для ускорения перебора предлагается создать триграммный индекс – для каждой буквенной триграммы храним список слов, в которых она есть. Тогда для поиска похожих на данное слово найдем слова большим количеством совпадающих триграмм. 

Совет 1: стоит сделать отельный индекс для каждой длинны слова и использовать только те индексы, в которых лежат слова близкие по длине к исходному.

Совет 2: для выделения триграмм стоит обрамить слово спецсимволом, чтобы триграммы на концах слова отличались от оных в середине.

Любые другие алгоритмы, улучшающие качество за разумное время (хождение по бору с ошибками, перебор ошибок) – не возбраняются.

Не побрезгуйте кешировать результат работы этого алгоритма, чтобы дальнейшая работа протекала быстрее.

In [7]:
from collections import Counter
from nltk import ngrams

In [8]:
dct = Counter()
words = []
for li, line in enumerate(read_huge_corpus("./drive/MyDrive/train.bz2")):
    words += line.split()

4717753it [00:48, 97758.63it/s]


In [9]:
dct = Counter([word.lower() for word in words])

In [10]:
del words

In [11]:
len(dct)

1681973

In [16]:
cnt = 0
for i in dct:
    if dct[i] <= 2:
        cnt += 1

In [17]:
cnt

1246761

In [18]:
dct = {word:cnt for word, cnt in dct.items() if cnt >= 3}

In [19]:
len(dct)

435212

In [20]:
def get_index(dct):
    index = dict()
    for word in tqdm(dct):
        groups = index.get(len(word), dict())
        for ngram in ngrams(f'${word}$', 3):
            tgram = ''.join(ngram)
            group = groups.get(tgram, [])
            group.append(word)
            groups[tgram] = group
        index[len(word)] = groups
    return index

In [21]:
index = get_index(dct)

100%|██████████| 435212/435212 [00:05<00:00, 83570.00it/s]


In [22]:
def extract_different_words(queries: Queries) -> List[Tuple[Word, Word]]:
    words_to_fix = []
    for original, fixed in queries:
        if original != fixed:
            for word_orig, word_fixed in zip(original.split(), fixed.split()):
                if word_orig != word_fixed:
                    words_to_fix.append((word_orig, word_fixed))
    return words_to_fix
                    
words_to_fix = extract_different_words(queries)
print(f'Found {len(words_to_fix)} words to fix')
for original, fixed in words_to_fix[:10]:
    print(diff_queries(original, fixed))

Found 53495 words to fix
chlothes
cataloges
compound
barnes
embearirassing
florirda
hdepot
inspection
biypass
theift


In [23]:
def find_similar_words(word: Word) -> List[Word]:
    size = len(word)
    trigrams = [''.join(ngram) for ngram in ngrams(f'${word}$', 3)]
    wrds = Counter()
    for wlen in range(max(size - 2, 1), size + 3):
        groups = index.get(wlen, {})
        for tgram in trigrams:
            wrds.update(Counter(groups.get(tgram, [])))
    
    wrds = [(wrd, cnt) for wrd, cnt in wrds.items()]
    wrds = sorted(wrds, key=lambda x: -x[1])
    
    return [wrd for wrd, cnt in wrds]


for original, fixed in words_to_fix[:5]:
    similar = find_similar_words(original)
    print(original, fixed, '- ok' if fixed in similar else '- fail')
    for word in similar[:5]:
        print(' ', word)
    print()

chothes clothes - ok
  rothes
  clothes
  soothes
  chota
  choti

cataloges catalogs - ok
  cataloged
  catalogues
  catalogers
  catalog
  catalogs

compond compound - ok
  compound
  component
  compo
  compose
  compost

barns barnes - ok
  barns
  barnens
  barn
  arns
  barne

emberissing embarrassing - ok
  embossing
  remembering
  embarrassing
  dismembering
  crisscrossing



Чтобы оценить качество полученного алгоритма, используйте запросы из `queries.tsv.gz`. Отберите только отличающиеся слова в исправленном и исходном запросах. Проверьте, что для слова в исходном запросе, исправленное слово будет в списке ближайших выданном вашим алгоритмом. Если это выполняется для всех или почти всех пар – успех. 

In [24]:
def check_find_similar_words(words_to_fix: List[Tuple[Word, Word]], 
                             find_similar_words: Callable[[Word], List[Word]], 
                             debug: bool):
    wrong, total = 0, 0
    progress = tqdm(words_to_fix)
    debug_output = 0
    for word_orig, word_fixed in progress:
        similar = find_similar_words(word_orig)
        if word_fixed not in similar:
            wrong += 1
            if debug:
                print(word_orig, word_fixed)
                debug_output += 1
                if debug_output == 10:
                    break
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')
        
check_find_similar_words(words_to_fix, find_similar_words, debug=False)

Wrong: 113 - 5.50%:   4%|▍         | 2056/53495 [00:35<14:52, 57.65it/s]


KeyboardInterrupt: ignored

## 3. Языковая модель
Языковая модель – модель, которая по тексту оценивает вероятность того, что он мог появиться в языке. 

Постройте простую n-грамную языковую модель с использованием корпуса текстов `train.bz2`. Для этого рассчитайте количество вхождений каждой n-граммы в корпус текста. Если взять n=2, то размера оперативной памяти вашего компьютера должно будет хватить.

Воспользуйтесь каким-нибудь методом сглаживания, чтобы не получать нулевую вероятность для неизвестных n-грамм. Также, чтобы вероятности слов, которых нет в словаре, были отличны от нуля, можно примешать побуквенную m-граммную модель.

Совет N: если количество оперативной памяти прижмёт, можно хранить строки в виде байт – один раскодированный символ занимает больше памяти чем один байт, при этом для английского текста почти всегда один символ кодируется одним байтом.

In [26]:
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

def get_model(sentences):
    unigrms = defaultdict(lambda: 0)
    bigrms = defaultdict(lambda: defaultdict(lambda: 0))
    ttl = 0
    ttlb = 0
    for sentence in tqdm(sentences):
        for w1 in sentence.split():
            unigrms[w1.lower()] += 1
            ttl += 1
        for w1, w2 in bigrams(sentence.split()):
            bigrms[w1.lower()][w2.lower()] += 1
            ttlb += 1

    for w1 in tqdm(unigrms):
        unigrms[w1.lower()] /= ttl
    for w1 in tqdm(bigrms):
        for w2 in bigrms[w1.lower()]:
            bigrms[w1.lower()][w2.lower()] /= ttlb
    return unigrms, bigrms

In [27]:
sentences = [line for line in read_huge_corpus("./drive/MyDrive/train.bz2")]

4717753it [00:35, 132536.88it/s]


In [28]:
unigrms, bigrms = get_model(sentences)

100%|██████████| 1500088/1500088 [00:12<00:00, 120478.24it/s]


In [47]:
unigrms = {word:cnt for word, cnt in unigrms.items() if word in dct}

In [50]:
bigrms = {word:cnt for word, cnt in bigrms.items() if word in dct}

In [53]:
def get_mgrams(sentences):
    mgrams = defaultdict(lambda: 0)
    ttl = 0
    for sentence in tqdm(sentences):
        for word in sentence.split():
            w = word.lower()
            if w in dct:
                for ngram in ngrams(f'${w}$', 3):
                    tgram = ''.join(ngram)
                    mgrams[tgram] += 1
                    ttl += 1
    for tgram in mgrams:
        mgrams[tgram] /= ttl
    return dict(mgrams)

In [54]:
tgrams = get_mgrams(sentences)

100%|██████████| 4717753/4717753 [07:22<00:00, 10666.90it/s]


In [56]:
cnt = 0
for tgram in tgrams:
    print(tgram, tgrams[tgram])
    if cnt > 10:
        break
    cnt += 1

$go 0.000543188731663703
gol 0.00010343088474842835
ol$ 0.00046026836305345834
$ne 0.0010225178800616233
nes 0.0005390644062852137
esh 9.98250762231679e-05
shi 0.0006792068133410809
hin 0.0005144295647685407
in$ 0.007964045850921527
$mi 0.0009877455049654947
mit 0.00022649927256582832
ito 0.00014170324860385237


In [81]:
def get_probability(query: Query) -> float:
    probability = 0
    words = [word.lower() for word in query.split()]
    for w1, w2 in bigrams(words):
        score = bigrms.get(w1, {}).get(w2, 0)
        if score > 0:
            probability += 1
        else:
            probability -= 1
            for w in (w1, w2):
                if w not in unigrms:
                    # for ngram in ngrams(f'${w}$', 3):
                    #     tgram = ''.join(ngram)
                    #     if tgram not in tgrams:
                    #         probability -= 1
                # else:
                    probability -= 1
                
    return probability

In [77]:
# def get_probability(query: Query) -> float:
#     probability = 0
#     words = [word.lower() for word in query.split()]
#     for w1, w2 in bigrams(words):
#         probability += bigrms.get(w1, {}).get(w2, 0)
#         probability += unigrms.get(w1, 0) + unigrms.get(w2, 0)
#         score = 1
#         # for ngram in ngrams(f'${w1}$', 3):
#         #     tgram = ''.join(ngram)
#         #     probability += tgrams.get(tgram, 0)
#     return probability

for original, fixed in queries_sample:
    p_original = get_probability(original)
    p_fixed = get_probability(fixed)
    verdict = '[ok]  ' if p_fixed > p_original else '[fail]'
    sign = '< ' if p_fixed > p_original else '>='
    print(f'{verdict} {original:>40s} {p_original:5.2f}  {sign} {p_fixed:5.2f} {fixed}')

[ok]                          grand theift auto  0.00  <   0.00 grand theft auto
[ok]             belarus longitude and latitdue  0.06  <   0.06 belarus longitude and latitude
[ok]                          search for poeoms  0.02  <   0.02 search for poems
[ok]      large guacolmoi dip restaurtant price  0.00  <   0.00 large guacamole dip restaurant price
[ok]                    texas chainsaw mascurer  0.00  <   0.00 texas chainsaw massacre
[fail]                     royal trump subtitle  0.00  >=  0.00 royal tramp subtitle
[ok]                   florida fiberglass polls  0.00  <   0.00 florida fiberglass pools
[ok]                     how to make a calender  0.08  <   0.08 how to make a calendar
[ok]               university of south caroline  0.07  <   0.07 university of south carolina
[fail]             maureen mcdonald in virginia  0.07  >=  0.07 maureen mcdonnell in virginia


Чтобы оценить качество полученной модели, используйте запросы из `queries.tsv.gz`. Сравните вероятность, которую выдает ваша модель для исходных и исправленных запросов. Хорошая модель выдаёт исправленному запросу большую вероятность. 

Советую сохранить полученную модель на диск – а случае чего, чтение статистик с диска, может быть быстрее расчёта оных с нуля.

In [ ]:
def check_language_model(queries: Queries, get_probability: Callable[[Query], float], debug: bool):
    wrong, total = 0, 0
    progress = tqdm(queries)
    debug_output = 0
    for original, fixed in progress:
        if original == fixed:
            continue
        p_original = get_probability(original)
        p_fixed = get_probability(fixed)
        if p_fixed <= p_original:
            wrong += 1
            if debug:
                print(original, p_original)
                print(fixed, p_fixed)
                print()
                debug_output += 1
                if debug_output == 10:
                    break
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')
        
check_language_model(queries, get_probability, debug=False)

Wrong: 2329 - 4.54%: 100%|██████████| 102436/102436 [01:57<00:00, 870.38it/s] 


In [82]:
def check_language_model(queries: Queries, get_probability: Callable[[Query], float], debug: bool):
    wrong, total = 0, 0
    progress = tqdm(queries)
    debug_output = 0
    for original, fixed in progress:
        if original == fixed:
            continue
        p_original = get_probability(original)
        p_fixed = get_probability(fixed)
        if p_fixed <= p_original:
            wrong += 1
            if debug:
                print(original, p_original)
                print(fixed, p_fixed)
                print()
                debug_output += 1
                if debug_output == 10:
                    break
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')
        
check_language_model(queries, get_probability, debug=False)

Wrong: 533 - 9.22%:  11%|█▏        | 11635/102436 [00:11<01:32, 977.23it/s] 


KeyboardInterrupt: ignored

### 4. Модель ошибок
Модель ошибок – модель которая по исходному и исправленному запросу оценивает вероятность того, что такая ошибка могла быть допущена.

Рассчитайте простую модель ошибок на основе расстояния Дамерау-Левенштейна, то есть модифицированного Левенштейна, который считает перестановку соседних букв за одну ошибку.

In [35]:
!pip install fastDamerauLevenshtein

In [36]:
from fastDamerauLevenshtein import damerauLevenshtein

In [37]:
import numpy as np

In [38]:
def get_error_probability(original: Query, fixed: Query) -> float:
    dist = damerauLevenshtein(original, fixed, similarity=False, 
                              deleteWeight=1,
                              insertWeight=1, 
                              replaceWeight=1,
                              swapWeight=1)   
    return 1 if dist == 0 else 1.3 ** -dist

for original, fixed in queries_sample:
    p_error = get_error_probability(original, fixed)
    print(f'{original:>40s} | {p_error:5.2f} | {fixed}')

                       grand theift auto |  0.77 | grand theft auto
          belarus longitude and latitdue |  0.77 | belarus longitude and latitude
                       search for poeoms |  0.77 | search for poems
   large guacolmoi dip restaurtant price |  0.27 | large guacamole dip restaurant price
                 texas chainsaw mascurer |  0.35 | texas chainsaw massacre
                    royal trump subtitle |  0.77 | royal tramp subtitle
                florida fiberglass polls |  0.77 | florida fiberglass pools
                  how to make a calender |  0.77 | how to make a calendar
            university of south caroline |  0.77 | university of south carolina
            maureen mcdonald in virginia |  0.46 | maureen mcdonnell in virginia


## 5. Всё вместе (1 балл)
Объедините результат работы предыдущих пунктов в единый алгоритм исправления опечатки для запроса.

Примерный план:
1.	Для слов запроса генерируем список ближайших слов-кандидатов (для всех, даже словарных слов).
2.	Собираем список кандидатов-запросов (эвристически, чтобы не сделать экспоненциальное время выполнения)
3.	Для каждого кандидата считаем итоговый объединенный score на основе языковой модели и модели ошибок для данного кандидата (не обязательно сумма или произведение, можно объединение любой сложности).
4.	Выдаём гипотезу с наибольшим score.
5.	???
6.	Profit

In [39]:
from itertools import product

In [79]:
def correct(query: Query) -> Query:
    words = query.split()
    similar_words = []
    for word in words:
        sim_words = find_similar_words(word)
        sim_words = [(w, get_error_probability(word, w)) for w in sim_words]
        sim_words = sorted(sim_words, key=lambda x: -x[1])
        similar_words.append([w for w, p in sim_words[:2]])
    best = (1000000000, '')
    for q in product(*similar_words):
        qr = ' '.join(q)
        cur = (-get_probability(qr), qr)
        if best[0] > cur[0]:
            best = cur
    return best[1]

for original, fixed in queries_sample:
    predict = correct(original)
    verdict = '[ok]  ' if predict == fixed else '[fail]'
    sign = '==' if predict == fixed else '!='
    print(f'{verdict} {predict:>40s} {sign} {fixed}')

[ok]                           grand theft auto == grand theft auto
[ok]             belarus longitude and latitude == belarus longitude and latitude
[ok]                           search for poems == search for poems
[fail]       large giacomo dip restaurant price != large guacamole dip restaurant price
[fail]                    texas chainsaw maurer != texas chainsaw massacre
[fail]                    royal trump subtitled != royal tramp subtitle
[fail]                  florida fiberglass poll != florida fiberglass pools
[fail]                  how to make a callender != how to make a calendar
[fail]             university of south caroline != university of south carolina
[fail]             maureen mcdonald in virginia != maureen mcdonnell in virginia


Итоговое качество меряем на примерах из `queries.tsv.gz`.

Для отладки проблем с качеством имеет смысл научится понимать на каком этапе теряется правильная гипотеза для каждого примера. Например, если правильное исправление есть в списке кандидатов (п. 2), но не выбирается как лучшая – стоит крутить языковую модель, модель ошибок и их объединение.

In [83]:
def check_corrector(queries: Queries, correct: Callable[[Query], Query], debug: bool):
    wrong, total = 0, 0
    progress = tqdm(queries)
    debug_output = 0
    for original, fixed in progress:
        predict = correct(original)
        if predict != fixed:
            wrong += 1
            if debug:
                print(original)
                print(fixed)
                print(predict)
                print()
                debug_output += 1
                if debug_output == 10:
                    break
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')
        
check_corrector(queries, correct, debug=False)

Wrong: 5931 - 29.23%:  20%|█▉        | 20291/102436 [34:40<2:20:23,  9.75it/s]


KeyboardInterrupt: ignored

In [84]:
import random

In [85]:
def check_corrector(queries: Queries, correct: Callable[[Query], Query], debug: bool):
    wrong, total = 0, 0
    progress = tqdm(queries)
    debug_output = 0
    for original, fixed in progress:
        predict = correct(original)
        if predict != fixed:
            wrong += 1
            if debug:
                print(original)
                print(fixed)
                print(predict)
                print()
                debug_output += 1
                if debug_output == 10:
                    break
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')
random.shuffle(queries)
check_corrector(queries, correct, debug=False)

Wrong: 1702 - 29.46%:   6%|▌         | 5777/102436 [10:04<2:48:27,  9.56it/s]


KeyboardInterrupt: ignored